In [1]:
import pandas as pd

# Define the bucket and file names
bucket_name = 'mimicivliza'  # Replace with your bucket name
mimic_iv_bhc = f's3://{bucket_name}/sample_data_100.csv'

# Load the files
mimic_iv_bhc_100 = pd.read_csv(mimic_iv_bhc)

# Display the data
mimic_iv_bhc_100.head(20)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/fsspec/registry.py:283: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


,note_id,input,target,input_tokens,target_tokens
0,16002318-DS-17,<SEX> F <SERVICE> SURGERY <ALLERGIES> Iodine /...,This is a ___ yo F admitted to the hospital af...,1195,75
1,15638884-DS-4,<SEX> M <SERVICE> MEDICINE <ALLERGIES> Augment...,Mr. ___ is a ___ yo man with CAD with prior MI...,3496,1143
2,12435705-DS-14,<SEX> M <SERVICE> MEDICINE <ALLERGIES> ibuprof...,Mr. ___ is a ___ w/ Ph+ve ALL on dasatanib and...,5591,1098
3,12413577-DS-4,<SEX> F <SERVICE> OBSTETRICS/GYNECOLOGY <ALLER...,"On ___, Ms. ___ was admitted to the gynecology...",1119,221
4,17967161-DS-29,<SEX> M <SERVICE> SURGERY <ALLERGIES> lisinopr...,Mr. ___ underwent an angiogram on ___ which sh...,3307,439
5,16956007-DS-20,<SEX> M <SERVICE> SURGERY <ALLERGIES> Codeine ...,Mr. ___ is a ___ who underwent an exploratory ...,4168,1209
6,16919911-DS-15,<SEX> F <SERVICE> MEDICINE <ALLERGIES> Penicil...,This is a ___ year old female with a recent di...,2059,208
7,15682570-DS-25,<SEX> M <SERVICE> MEDICINE <ALLERGIES> No Know...,"___ w/ h/o CAD ___ CABG LIMA to LAD, SVG to D1...",2215,451
8,12135369-DS-24,<SEX> F <SERVICE> MEDICINE <ALLERGIES> Compazi...,Ms ___ is a ___ year old woman with a history ...,2132,416
9,11906321-DS-20,<SEX> M <SERVICE> NEUROSURGERY <ALLERGIES> Pat...,The patient was admitted to the neurosurgery s...,2347,316


In [2]:
# Install Hugging Face Transformers
!pip install -q transformers
!pip install -q sacremoses
!pip install -q bitsandbytes accelerate

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,           # or load_in_4bit=True for more compression
    llm_int8_threshold=6.0,      # default threshold for 8-bit
    llm_int8_skip_modules=None,  # or specify modules to skip
    bnb_8bit_compute_dtype="float16"  # use float16 for faster inference
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/BioGPT-Large")
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/BioGPT-Large",
    quantization_config=bnb_config,
    device_map="auto"  # use this if you're on a GPU or want automatic device placement
)



tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/566k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.29G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [56]:
from tqdm import tqdm
import torch

# Adjusted prompt with clearer instructions
def construct_few_shot_prompt(input_text):
    prompt = (
        "You are an expert clinician. Summarize the patient case concisely in a storytelling manner, "
        "Ensure the summary does not exceed 200 words and avoids directly copying any text.\n\n"
        "Example - Input: A 50-year-old male reports severe chest pain and has a history of heart disease.\n"
        "Example - Summary: A male patient with heart disease experienced severe chest pain and received emergency angioplasty.\n\n"
        f"Input: {input_text}\nSummary:"
    )
    return prompt

subset_df = mimic_iv_bhc_100.iloc[:101].copy()  # Processing the first 101 entries for the example
generated_summaries = []

for input_text in tqdm(subset_df['input'], desc="Generating Summaries with BioGPT-Large"):
    prompt = construct_few_shot_prompt(input_text)

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1000  # Adjust max length according to your model's capabilities
    )

    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    generation_params = {
        "do_sample": False,
        "max_new_tokens": 200,
        "temperature": 0.5,
        "top_p": 0.8,
        "top_k": 20,
        "repetition_penalty": 1.2,
        "eos_token_id": tokenizer.eos_token_id
    }

    with torch.no_grad():
        original_length = inputs["input_ids"].shape[1]
        summary_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            **generation_params
        )

        # Decode only the newly generated tokens
        generated_tokens = summary_ids[0, original_length:]
        generated_summary = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

    generated_summaries.append(generated_summary)
    torch.cuda.empty_cache()  # Optional: Clear CUDA cache after each generation to free memory

# Add generated summaries to DataFrame and save
subset_df['generated_summary_biogpt_large'] = generated_summaries
subset_df.to_csv("biogpt_large_generated_summaries.csv", index=False)
print("✅ Subset summaries saved to 'biogpt_large_generated_summaries.csv'")


Generating Summaries with BioGPT-Large:   0%|          | 0/100 [00:00<?, ?it/s]/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation 

✅ Subset summaries saved to 'biogpt_large_generated_summaries.csv'


In [53]:
subset_df.head()

,note_id,input,target,input_tokens,target_tokens,generated_summary_biogpt_large
0,16002318-DS-17,<SEX> F <SERVICE> SURGERY <ALLERGIES> Iodine /...,This is a ___ yo F admitted to the hospital af...,1195,75,activities until you have been off these drugs...
1,15638884-DS-4,<SEX> M <SERVICE> MEDICINE <ALLERGIES> Augment...,Mr. ___ is a ___ yo man with CAD with prior MI...,3496,1143,"Abdomen: No tenderness, no guarding, no palpab..."


In [54]:
print(subset_df['input'].iloc[1])

<SEX> M <SERVICE> MEDICINE <ALLERGIES> Augmentin / Nsaids / Tramadol / Niacin <ATTENDING> ___ ___ Complaint: Painless jaundice <MAJOR SURGICAL OR INVASIVE PROCEDURE> ERCP with stent and sphincterotomy on ___ <HISTORY OF PRESENT ILLNESS> Per Dr. ___ note: "Mr. ___ is a ___ male with a PMH of CAD s/p MI, ischemic CMY, HTN, alcohol abuse, pancreatic insufficiency who presents with painless jaundice. He has had intermittent white stools for 6 months. RUQ US with pancreatic head mass, choledolithiasis and enlarging abdominal aorta. He slipped and fell on his daughter's wet steps two weeks ago and since then noted that his urine was yellow-> dark. His family told him that he was turning yellow at a family BBQ 3 days ago. His family also noted that his eyes were yellow. He saw his PCP two days ago who recommended an US. He was found to have a "cyst" and he was advised to go to the ED. He does not have any abdominal pain. He has chronic L shoulder pain. He does not have chest pain or shortness

In [55]:
print(subset_df['generated_summary_biogpt_large'].iloc[1])

Abdomen: No tenderness, no guarding, no palpable liver edge, no ascites, no splenomegaly, no bowel sounds. Chest: Normal lung fields, normal breathing sounds, no rales, no jugular venous pressure. Abdomen: No hepatomegaly, no splenomegaly, no percussion tenderness, no dullness to percussion. Thorax: Normal lungs, no pleural effusion, no atelectasis, no lymphadenopathy, no rib notching, no scoliosis. Pelvis: Normal ribs, no scoliosis, no lordosis. < / FREETEXT > < / ABSTRACT > ▃


In [5]:
from tqdm import tqdm

# Few-shot examples for prompt construction
few_shot_examples = [
    {
        "input": "<SEX> M <SERVICE> CARDIOLOGY <CHIEF COMPLAINT> chest pain <HISTORY OF PRESENT ILLNESS> A 55-year-old male presented with chest pain radiating to his left arm and jaw. He reported associated shortness of breath and nausea. Initial ECG showed ST-segment elevation in the inferior leads, and troponin levels were elevated, consistent with an acute myocardial infarction. The patient was emergently taken to the catheterization lab, where coronary angiography revealed 100% occlusion of the right coronary artery. A drug-eluting stent was successfully placed, and dual antiplatelet therapy was initiated. He was monitored in the cardiac care unit for 48 hours with no complications.",
        "Summary": "A 55-year-old male with a myocardial infarction underwent PCI with stenting of the right coronary artery. He was monitored and started on dual antiplatelet therapy."
    },
    {
        "input": "<SEX> M <SERVICE> NEUROSURGERY <CHIEF COMPLAINT> headache past ten days <HISTORY OF PRESENT ILLNESS> The patient is a 60-year-old male with a history of renal cell carcinoma, presenting with progressively worsening right-sided headaches over 10 days. Imaging revealed a large intracranial mass in the right temporoparietal lobe with associated hemorrhage, necrosis, and vasogenic edema. He was started on Decadron to reduce cerebral edema and transferred for neurosurgical evaluation. On hospital day 5, the patient underwent an image-guided right craniotomy for tumor resection. Postoperative MRI showed no significant residual tumor. He recovered without complications and was discharged home on a tapering dose of Decadron, with follow-up in the brain tumor clinic.",
        "Summary": "A 60-year-old male with an intracranial mass underwent a craniotomy for tumor resection. He recovered well and was discharged with follow-up."
    },
    {
        "input": "<SEX> F <SERVICE> GENERAL MEDICINE <CHIEF COMPLAINT> shortness of breath <HISTORY OF PRESENT ILLNESS> A 72-year-old female with a history of COPD presented with progressive shortness of breath over two weeks. Physical examination revealed decreased breath sounds and crackles in the left lower lobe. Imaging confirmed left lower lobe consolidation consistent with pneumonia. The patient was started on broad-spectrum antibiotics and oxygen therapy. Arterial blood gas analysis showed mild hypoxemia, which improved with oxygen supplementation. She remained afebrile throughout her stay and reported gradual improvement in symptoms. Discharge planning included antibiotics, a pulmonary follow-up, and instructions for home oxygen therapy.",
        "Summary": "A 72-year-old female with COPD was treated for pneumonia with antibiotics and oxygen therapy. She showed improvement and was discharged with follow-up."
    }
]

# Function to construct the prompt (unchanged)
def construct_few_shot_prompt(input_text):
    prompt = "You are a medical expert. Please summarize the following input concisely  and cohesively:\n\n. Do not include anything from the exmple."
    for example in few_shot_examples:
        prompt += f"Input: {example['input']}\nTarget: {example['target']}\n\n"
    prompt += f"Input: {input_text}\nSummary:"
    return prompt

# Step 2: Generate Summaries with BioGPT-Large
generated_summaries = []

for input_text in tqdm(mimic_iv_bhc_100['input'], desc="Generating Summaries with BioGPT-Large"):
    # Construct the prompt
    prompt = construct_few_shot_prompt(input_text)

    # Tokenize prompt for BioGPT (causal language model)
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        max_length=1024,
        truncation=True
    )
    
    # Move tokenized input tensors to model's device (e.g., CUDA)
    device = model.device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate the summary using causal generation
    generation_params = {
        "do_sample": True,
        "top_p": 0.8,
        "temperature": 0.7,
        "top_k": 40,
        "max_new_tokens": 150,
        "repetition_penalty": 1.1
    }

    summary_ids = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        **generation_params
    )

    # Decode and post-process the output
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    generated_summary = generated_summary.split("Summary:")[-1].strip()  # Optional cleanup
    generated_summaries.append(generated_summary)

# Step 3: Add Generated Summaries to the DataFrame
mimic_iv_bhc_100['generated_summary_biogpt_large'] = generated_summaries

# Optional: Save the results
mimic_iv_bhc_100.to_csv("biogpt_large_generated_summaries.csv", index=False)
print("Summaries generated and saved to 'biogpt_large_generated_summaries.csv'")



Generating Summaries with BioGPT-Large: 100%|██████████| 100/100 [32:18<00:00, 19.38s/it]

Summaries generated and saved to 'biogpt_large_generated_summaries.csv'
